In [ ]:
import numpy as np
from view_sampler import *
from algs import *

import loss_funcs
from evaluate import eval_funcs

from evaluate.eval_log import EvalLog
from evaluate.evaluator import Evaluator
from evaluate.dataset import Dataset
from utils.visualize import *

import mealpy

plotter = SearchPlotter(update_freq=100, history=None, alpha=0.5)

# Create a camera configuration
cam_config = CameraConfig(location=(0, 0, 0.3), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/hammer/world.xml", cam_config, simulation_time=0)
sim_viewer = ViewSampler("data/hammer/world_sim.xml", cam_config)

loss_func = loss_funcs.IOU()

algorithms = [
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.PSO.OriginalPSO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.MSA.OriginalMSA()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.SCSO.OriginalSCSO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.SA.OriginalSA()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.EVO.OriginalEVO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.EFO.DevEFO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.EO.ModifiedEO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.human_based.ICA.OriginalICA(revolution_prob=0.4, empire_count=7)),
    MealAlgorithm(sim_viewer, loss_func, mealpy.human_based.FBIO.DevFBIO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.human_based.SARO.OriginalSARO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.evolutionary_based.GA.BaseGA()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.evolutionary_based.CRO.OCRO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.evolutionary_based.DE.OriginalDE()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.math_based.PSS.OriginalPSS()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.math_based.SCA.DevSCA()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.math_based.HC.OriginalHC()),
]

from mealpy.tuner import Tuner

eval_func = eval_funcs.XorDiff(0.1)
evaluator = Evaluator(world_viewer, sim_viewer, eval_func=eval_func)

dataset = Dataset.create_random(location=(0, 1.3, 0.3), num_samples=100, seed=1)
# dataset.visualize(sim_viewer)
run_config = MealRunConfig(time_limit=15, silent=True)

for alg in algorithms:
    log = EvalLog(alg)
    eval_losses = evaluator.evaluate(alg, run_config, dataset, log=None, plotter=None)
    log.save("logs/")
    print(f"{alg.get_name()} {alg.loss_func.get_name()}: {eval_losses}")

In [ ]:
world_viewer.close()
sim_viewer.close()
plotter.close()

# ANALYSIS

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
from evaluate.eval_log import MealLog
from mealpy.utils.history import History
from utils.file import LogFiles
import pandas as pd

hs_path = "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Robotics/Robotics-Proj/Ealuations/UniformDet/Mug/IOU/DevHS_20240318-191420.pickle"
pso_path = "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Robotics/Robotics-Proj/Ealuations/UniformDet/Mug/IOU/OriginalPSO_20240318-194706.pickle"
bro_path = "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Robotics/Robotics-Proj/Ealuations/UniformDet/Mug/IOU/DevBRO_20240318-201942.pickle"

path = "Evals/OverNight/UniformDet/Hammer/IOU"

# logHS = MealLog.load(hs_path)
# logPSO = MealLog.load(pso_path)
# logBRO = MealLog.load(bro_path)

files = LogFiles(path)

In [ ]:
data = []
for _ in files:
    data.append(files.to_dataframe())

dataframe = pd.concat(data, axis=0, ignore_index=True)

In [ ]:
dataframe.columns

In [ ]:
algs = pd.unique(dataframe["alg"])
medians = {"alg": [], "median": []}
for alg in algs:
    mask = dataframe["alg"] == alg
    # print(dataframe[mask][['eval_loss']])
    median = dataframe[mask][["eval_loss"]].median()
    # medians[alg] = median.item()
    medians["alg"].append(alg)
    medians["median"].append(median.item())

In [ ]:
medians = pd.DataFrame(medians)
medians.sort_values(by="median", inplace=True)

In [ ]:
# fig, ax = plt.subplots(figsize=(10,40))
g = sns.catplot(
    data=dataframe,
    x="alg",
    y="eval_loss",
    hue="alg",
    kind="box",
    errorbar="sd",
    aspect=8,
    order=medians.alg,
    hue_order=medians.alg,
)
g.ax.tick_params(axis="x", rotation=90)

In [ ]:
print(medians.alg.to_list())